#### FISH Image Converter (v 1.0)

### Input Image info:

In [1]:
#specify where the directory of the data
local_download_directory = '/scratch/alpine/naly@colostate.edu/bigfish/erm-1_ERM-1-png-test/280829_erm-1_ERM-1' 

#specify what you have in every channel
Cy5 = "set-3 mRNA" #(asymmetrical control)
mCherry = "erm-1 mRNA" #(query mRNA)
FITC = "membrane" #(ignore in this case) This is where protein markers usually go.
DAPI = "DAPI"


In [2]:
# Import packages
import os
import shutil
import bigfish.stack as stack

#import packages:
import os
import matplotlib.pyplot as plt
import numpy as np

import tifffile
import bigfish
import bigfish.stack as stack
import bigfish.plot as plot
import bigfish.multistack as multistack
import bigfish.detection as detection


#### 1.2 Read images

In [3]:
#Specify where the data will be stored
folder_name = local_download_directory.split(os.path.sep)[-1]
input_directory = os.path.join(local_download_directory, 'input')
output_directory = os.path.join(local_download_directory, 'images')

# Get a list of subdirectories within local_download_directory
subdirectories = [item for item in sorted(os.listdir(input_directory)) if item != '.ipynb_checkpoints' if os.path.isdir(os.path.join(input_directory, item))]

subdirectory_paths = []
for subdirectory in subdirectories:
    subdirectory_path = os.path.join(local_download_directory, subdirectory)
    subdirectory_paths.append(subdirectory_path)

# Iterate through the subdirectories and store their path in a list
input_subdirectory_paths = []
for subdirectory in subdirectories:
    input_subdirectory_path = os.path.join(input_directory, subdirectory)
    input_subdirectory_paths.append(input_subdirectory_path)

# Iterate through each subdirectory path
all_image_stacks = []

for input_subdirectory_path in input_subdirectory_paths:
    files = [os.path.join(input_subdirectory_path, item) for item in os.listdir(input_subdirectory_path)]
    
    # Check if exactly 2 files with the specified names are present
    if len(files) == 2 and any(file.endswith("R3D_REF.dv") for file in files) and any(file.endswith("R3D.dv") for file in files):
        files = sorted(files)
        
        # Store all image stacks in a list
        subdirectory_image_stacks = []
        for file_path in files:
            image_stack = stack.read_dv(file_path, sanity_check=False)
            subdirectory_image_stacks.append(image_stack)
        all_image_stacks.append(subdirectory_image_stacks)
        
    else:
        print(f"corrupted: {subdirectory_path}")
        # Copy the entire corrupted directory to the 'corrupted_files' directory
        corrupted_path = os.path.join(local_download_directory, 'corrupted_files')
        corrupted_subdirectory = os.path.basename(subdirectory_path)
        corrupted_destination = os.path.join(corrupted_path, corrupted_subdirectory)
        shutil.move(subdirectory_path, corrupted_destination)
        continue

# Now, all_image_stacks contains the image data from all the .dv files, organized by subdirectory.
print(f'Images read: {len(all_image_stacks)}')


Images read: 16


In [4]:
subdirectories

['01',
 '02',
 '03',
 '04',
 '05',
 '06',
 '07',
 '08',
 '09',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16']

In [5]:
#SAve images as png
all_channel_list = []

# Iterate through the subdirectories and store their path in a list
output_subdirectory_paths = []
for subdirectory in subdirectories:
    os.makedirs(output_directory, exist_ok=True)
    output_subdirectory_path = os.path.join(output_directory, subdirectory)
    os.makedirs(output_subdirectory_path, exist_ok=True)
    output_subdirectory_paths.append(output_subdirectory_path)

# Now iterate through all_image_stacks and extract the images
for i, stack in enumerate(all_image_stacks):
    image_colors = stack[0]

    # Assuming image_colors is a 4D array where the first dimension represents different channels
    for channel_index in range(image_colors.shape[0]):
        current_image = image_colors[channel_index, :, :]
        all_channel_list.append(current_image)

        # Save each image as a PNG file
        channel_name = None
        if channel_index == 0:
            channel_name = Cy5
        elif channel_index == 1:
            channel_name = mCherry
        elif channel_index == 2:
            channel_name = FITC
        elif channel_index == 3:
            channel_name = DAPI

        # Save each image as a PNG file
        plot_filename = os.path.join(output_subdirectory_paths[i], f"{subdirectories[i]}_{channel_name}.png")
        plt.imsave(plot_filename, np.max(current_image, axis=0), cmap='gray')  # Adjust cmap if needed


# For example, you can use the images in the plot function
def plot(all_channel_list):
    # Create subplots for each image in the list
    num_images = len(all_channel_list)
    fig, ax = plt.subplots(1, num_images, figsize=(24, 5))

    for i, image in enumerate(all_channel_list):
        ax[i].imshow(np.max(image, axis=0))
        ax[i].set_title(f"Image {i}")
        ax[i].axis('off')

print(f"Images in {output_directory}")


Images in /scratch/alpine/naly@colostate.edu/bigfish/erm-1_ERM-1-png-test/280829_erm-1_ERM-1/images


In [6]:
#Need to also save each brightfield as a png. 

